## Function Calling

GPT 3.5 ve 4 modelleri, Haziran-Eylül 2021 ile Aralık 2023 ve öncesinden gelen verilerle eğitildiği için maalesef, bu modeller Haziran-Eylül 2021 veya Aralık 2023'den sonra ortaya çıkan ya da keşfedilen bilgileri bilemezler. Ayrıca, karmaşık matematik problemlerini çözmek için yetersizdirler. Bu durumda, modellerimiz, tanımadıkları veya zayıf oldukları konularda daha tutarlı ve doğru sonuçlar döndürebilmek için model için tanımladığımız fonksiyonu çağırarak kullanılabilir.

Örneğin, vücut kitle indeksi hesaplaması Nisan 2023'den sonra keşfedilmiş olsun veya modeller vücut kitle indeksini hesaplama konusunda çok kötü olsun. Modelin vücut kitle indeksini doğru bir şekilde hesaplaması için aşağıda bir fonksiyon oluşturalım ve sonra bu fonksiyonu GPT modellerinde nasıl çağırdığımıza bakalım.

Ayrıca bir text'den istediğimiz verileri çekmek içinde Function calling çok kullanışlıdır. Langchain dersinde örneğini göreceğiz.






In [1]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY']=userdata.get('openai_apikey')

In [3]:
from openai import OpenAI

client = OpenAI()

In [4]:
def body_mass_index(height,  weight):
  "calculate the body mass index"
  b_m_i=weight/(height*height)

  return f"The body mass index is about {b_m_i:.2f}"

# First, we define our function to calculate the body mass index.

In [5]:
body_mass_index_func = {
    "name": "body_mass_index",
    "description": "Calculates the body mass index.",
    "parameters": {
        "type": "object",
        "properties": {
            "height": {
                "type": "number",
                "description": "person's height in meters"
            },
            "weight": {
                "type": "number",
                "description": "person's weight in kilograms"
            }

        },
        "required": ["height",  "weight"]
    }
}

# Fonksiyonun adını, açıklamasını (kullanım amacını), fonksiyon parametrelerini ve türlerini(type) yukarıda gösterildiği gibi tanımlıyoruz. Burada
# en önemli kısım, yapılan açıklamalardır(description). Çünkü model, bir fonksiyonu çağırıp çağırmama kararını bu fonksiyon açıklamalarına dayalı olarak
# verir. Ayrıca, parametre açıklamalarına dayalı olarak model, metinden ilgili parametreleri seçebilir.

# Modelle sorduğunuz soru, fonksiyonun açıklamasına semantik(anlamsal) olarak çok yakınsa, model bu fonksiyonu çağıracaktır. Ancak, fonksiyon açıklaması
# kötü yapılmış veya eksikse, model fonksiyonu çağırmayacaktır.

# Model, fonksiyonu çağırdıktan sonra, doğru parametreleri metinden seçmek için iyi tanımlanmış parametre açıklamalarına sahip olmak önemlidir.
# Eğer parametre açıklamaları iyi yapılmamışsa veya eksikse, model textden doğru parametreleri seçemez.

In [6]:
type(body_mass_index_func)

dict

In [7]:
prompt = "What is the body mass index of a person who is 1800 mm tall and weight 80000 grams?"

res = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{"role": "user", "content": prompt}],
    functions=[body_mass_index_func],
    function_call="auto" # "none"
)
res

# 'functions' parametresini 'body_mass_index_func' olarak ayarlayın.

# 'function_call' parametresini 'auto' olarak ayarlayın.

ChatCompletion(id='chatcmpl-A60CfGANKXTBhh3U1FFdRDXDBkMnK', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{"height":1.8,"weight":80}', name='body_mass_index'), tool_calls=None))], created=1725994113, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=87, total_tokens=108))

In [8]:
res.choices[0].message.function_call.arguments
# Prompt metninden seçilen parametreleri çekiyoruz.

# Gördüğünüz gibi, çıktımız bir JSON formatındadır (string).
# Python'da, JSON formatındaki veriyi doğrudan kullanamayız. Bu nedenle, json.loads() fonksiyonunu kullanarak veriyi dictionary formatına çevirerek
# Python'da JSON formatındaki veriyi kullanabiliriz.
# Veri dictionary formatına çevrildikten sonra, kolayca boy ve kilo bilgilerini çekebiliriz.

'{"height":1.8,"weight":80}'

In [9]:
import json

name = res.choices[0].message.function_call.name
arguments = json.loads(res.choices[0].message.function_call.arguments) # Bir JSON dizesini bir dictionary çevirmek için json.loads() fonksiyonunu kullanın
                                                                       # Bu fonksiyon, JSON string'i bir dictionarye çevirir.
print(name)
print(arguments)

body_mass_index
{'height': 1.8, 'weight': 80}


In [10]:
print(arguments["height"])
print(arguments["weight"])

# boy ve kilo bilgilerini kolayca çekebildik

1.8
80


In [11]:
body_mass_index(arguments["height"],  arguments["weight"])

# tanımladığımız fonksiyon içine parametreleri yazınca istediğim outputu döndürüyor.

'The body mass index is about 24.69'

In [12]:
def run_conversation(prompt):

    response = client.chat.completions.create(model='gpt-3.5-turbo',
                                              messages=[{"role": "user", "content": prompt}],
                                              functions=[body_mass_index_func],
                                              function_call="auto")


    # Step 1, modelin bir fonksiyon çağırmak(function_calling) isteyip istemediğini kontrol edin
    if  response.choices[0].finish_reason == "function_call":
        # json string'i dictionary'e çevirin
        arguments = json.loads(response.choices[0].message.function_call.arguments)

        # Step 2, fonksiyonu çağrın ve bir değişkene atayın
        function_response = body_mass_index(arguments["height"],
                                            arguments["weight"])

        return function_response # fonksiyon sonucunu döndürün
    return response.choices[0].message.content # eğer fonksiyon çağrılmıyorsa. Chatgpt'nin kendi cevabını döndür.


# run_conversation fonksiyonuna verdiğimiz textden sonra model "finish_reason" olarak "function_call" döndürüyorsa,
# cevaplarını body_mass_index fonksiyonu üzerinden verecektir. Aksi halde model cevaplarını kendi yöntemiyle verecektir.

In [13]:
run_conversation("What is the body mass index of a person who is 1800 mm tall and weight 80000 grams?")

'The body mass index is about 24.69'

In [14]:
run_conversation("What is the Capital of Spain?")

'The capital of Spain is Madrid.'